In [1]:
##########################################################複製區塊
import matplotlib.pyplot as plt
import time
from scipy import signal
from time import sleep
import sys
import meep as mp
import numpy as np
import random
import multiprocessing as multi
#setup some constants and parameters
M=2**16
tsim=7e-12
eps0 = 8.854187e-12
hbar = 1.05457182e-34
c = 2.99792458e8
kb = 1.380649e-23
hdk=7.63823258e-12
deltap = np.sqrt(1/3/eps0/hbar/c)*kb
T=3000
frq_min = 0
frq_max = 100/2/np.pi
nfreq = 1000
fcen = (frq_min + frq_max)/2
df = frq_max - frq_min
#define square root of Dn function
def Dnsqt(wla,T):
    return np.sqrt(6*hdk**2*wla/(np.exp(hdk*wla/T)-1)/T**2/np.pi)

#generate an array of the square root of Dn function
Dsqt = []
for n in range(1,M):
    Dsqt.append(Dnsqt(2*np.pi*n/tsim,T))

#define how many times to run to average the results
Ncomp= 20
nfreq =  1000
#define the starting sum of the result
Efsum = np.zeros(nfreq)
Ens = np.zeros(2**16)
resolution = 50
dpml = 1.0
cell = mp.Vector3(4,4,0)
pml_layers = [mp.PML(dpml)]



for i in range(Ncomp):
    
    def random_En(t):
        #generate random numbers
        M0 = np.random.normal(0,1)
        Mlp = np.random.normal(0,1,M-1)
        Nlp = np.random.normal(0,1,M-1)
        MM = np.random.normal(0,1)
    
        #generate the array of the fourier transform of E field, we generate l=0 and l=-M separately. 
        #for l=1 to M-1 and l=-1 to -(M-1) are conjutated to each other. 
        #we contruct the whole array by gluing them together by the order from l=0 -> l=M-1 -> l=-M -> l=-(m-1) -> l=-1
        Ef0 = np.array([M0*np.sqrt(6*hdk/np.pi/T)])
        Eflp = np.multiply((Mlp + 1j*Nlp), Dsqt)
        Eflm = np.flip(np.conjugate(Eflp))
        EfM = np.array([MM*Dnsqt(2*np.pi*M/tsim,T)])
        Ef = np.hstack((Ef0,Eflp,EfM,Eflm))
    
        #we normalize Ef and compute its inverse fast fourier transform
        Efn = deltap*T/np.sqrt(tsim)*Ef 
        En = np.fft.ifft(Efn)*M
        return En
    En0 = random_En(1)
    En1 = random_En(1)
    #generate some feedback to check the progress of the loop
    x = i/Ncomp*100
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('a'*int(x), int(x)))
    sys.stdout.flush()
    sleep(0.0001)
    
    
    def source0(t):
        f = int(t*20)
        return En0[f]
    def source1(t):
        f = int(t*20)
        return En1[f]
    


    sources = [mp.Source(mp.CustomSource(src_func=source0),
                     component=mp.Ex,
                     center=mp.Vector3(0,0,0),
                     ),
              mp.Source(mp.CustomSource(src_func=source1),
                     component=mp.Ex,
                     center=mp.Vector3(0,0,0),
                     )]
    
    geometry = [mp.Block(mp.Vector3(mp.inf,1,0),
                     center=mp.Vector3(),
                     material=mp.Medium(epsilon=1))]
    sim = mp.Simulation(cell_size=cell,
                    boundary_layers=pml_layers,
                    sources=sources,
                    geometry=geometry,   
                    dimensions = 2,
                    resolution=resolution)
    
    

    # transmitted flux
    tran_fr = mp.FluxRegion(center=mp.Vector3(random.uniform(-1,1),random.uniform(-1,1),0), size=mp.Vector3(2,0,0))
    tran = sim.add_energy(fcen, df, nfreq, tran_fr)
    
    
    sim.run(until=2090)

    tran_flux = mp.get_magnetic_energy(tran)
    Ts = []
    for i in range(nfreq):
        Ts = np.append(Ts,tran_flux[i])    
    Efsum += Ts
    
    

#average the final result
Efavg = Efsum/Ncomp

#save data to some file or reload file to write more data on it



[                    ] 0%-----------
Initializing structure...
time for choose_chunkdivision = 0.000319004 s
Working in 2D dimensions.
Computational cell is 4 x 4 x 0 with resolution 50
     block, center = (0,0,0)
          size (1e+20,1,0)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.124319 s
-----------


FloatProgress(value=0.0, description='0% done ', max=2090.0)

Meep progress: 18.25/2090.0 = 0.9% done in 4.0s, 454.1s to go
on time step 1828 (time=18.28), 0.00218942 s/step
Meep progress: 37.39/2090.0 = 1.8% done in 8.0s, 439.3s to go
on time step 3742 (time=37.42), 0.00209004 s/step
Meep progress: 56.65/2090.0 = 2.7% done in 12.0s, 430.8s to go
on time step 5669 (time=56.69), 0.00207677 s/step
Meep progress: 76.0/2090.0 = 3.6% done in 16.0s, 424.1s to go
on time step 7604 (time=76.04), 0.00206765 s/step
Meep progress: 95.37/2090.0 = 4.6% done in 20.0s, 418.4s to go
on time step 9541 (time=95.41), 0.00206533 s/step
Meep progress: 114.73/2090.0 = 5.5% done in 24.0s, 413.3s to go
on time step 11478 (time=114.78), 0.00206572 s/step
Meep progress: 134.1/2090.0 = 6.4% done in 28.0s, 408.5s to go
on time step 13417 (time=134.17), 0.00206319 s/step
Meep progress: 153.41/2090.0 = 7.3% done in 32.0s, 404.1s to go
on time step 15349 (time=153.49), 0.00207079 s/step
Meep progress: 172.82/2090.0 = 8.3% done in 36.0s, 399.5s to go
on time step 17291 (time=17

FloatProgress(value=0.0, description='0% done ', max=2090.0)

Meep progress: 19.62/2090.0 = 0.9% done in 4.0s, 422.2s to go
on time step 1963 (time=19.63), 0.00203851 s/step
Meep progress: 39.1/2090.0 = 1.9% done in 8.0s, 419.7s to go
on time step 3912 (time=39.12), 0.0020534 s/step
Meep progress: 58.72/2090.0 = 2.8% done in 12.0s, 415.2s to go
on time step 5874 (time=58.74), 0.00203902 s/step
Meep progress: 78.33/2090.0 = 3.7% done in 16.0s, 411.0s to go
on time step 7835 (time=78.35), 0.00204037 s/step
Meep progress: 97.82000000000001/2090.0 = 4.7% done in 20.0s, 407.4s to go
on time step 9784 (time=97.84), 0.00205245 s/step
Meep progress: 117.47/2090.0 = 5.6% done in 24.0s, 403.1s to go
on time step 11749 (time=117.49), 0.00203616 s/step
Meep progress: 136.88/2090.0 = 6.5% done in 28.0s, 399.6s to go
on time step 13690 (time=136.9), 0.002061 s/step
Meep progress: 156.37/2090.0 = 7.5% done in 32.0s, 395.8s to go
on time step 15640 (time=156.4), 0.0020523 s/step
Meep progress: 175.89000000000001/2090.0 = 8.4% done in 36.0s, 391.9s to go
on time 

...

Meep progress: 20.03/2090.0 = 1.0% done in 4.0s, 413.5s to go
on time step 2004 (time=20.04), 0.00199651 s/step
Meep progress: 40.160000000000004/2090.0 = 1.9% done in 8.0s, 408.5s to go
on time step 4017 (time=40.17), 0.00198752 s/step
Meep progress: 60.230000000000004/2090.0 = 2.9% done in 12.0s, 404.5s to go
on time step 6024 (time=60.24), 0.00199365 s/step
Meep progress: 80.37/2090.0 = 3.8% done in 16.0s, 400.2s to go
on time step 8038 (time=80.38), 0.0019864 s/step
Meep progress: 100.46000000000001/2090.0 = 4.8% done in 20.0s, 396.2s to go
on time step 10047 (time=100.47), 0.00199144 s/step
Meep progress: 120.5/2090.0 = 5.8% done in 24.0s, 392.4s to go
on time step 12051 (time=120.51), 0.00199622 s/step
Meep progress: 140.67000000000002/2090.0 = 6.7% done in 28.0s, 388.1s to go
on time step 14068 (time=140.68), 0.00198369 s/step
Meep progress: 160.72/2090.0 = 7.7% done in 32.0s, 384.3s to go
on time step 16073 (time=160.73), 0.0019956 s/step
Meep progress: 180.72/2090.0 = 8.6% don

FloatProgress(value=0.0, description='0% done ', max=2090.0)

Meep progress: 19.52/2090.0 = 0.9% done in 4.0s, 424.4s to go
on time step 1953 (time=19.53), 0.00204872 s/step
Meep progress: 39.08/2090.0 = 1.9% done in 8.0s, 419.9s to go
on time step 3909 (time=39.09), 0.00204518 s/step
Meep progress: 58.72/2090.0 = 2.8% done in 12.0s, 415.2s to go
on time step 5873 (time=58.73), 0.002037 s/step
Meep progress: 77.95/2090.0 = 3.7% done in 16.0s, 413.1s to go
on time step 7797 (time=77.97), 0.00208017 s/step
Meep progress: 97.17/2090.0 = 4.6% done in 20.0s, 410.2s to go
on time step 9720 (time=97.2), 0.00208075 s/step
Meep progress: 116.72/2090.0 = 5.6% done in 24.0s, 405.8s to go
on time step 11676 (time=116.76), 0.00204597 s/step
Meep progress: 136.3/2090.0 = 6.5% done in 28.0s, 401.4s to go
on time step 13634 (time=136.34), 0.00204335 s/step
Meep progress: 155.9/2090.0 = 7.5% done in 32.0s, 397.1s to go
on time step 15594 (time=155.94), 0.002041 s/step
Meep progress: 175.48/2090.0 = 8.4% done in 36.0s, 392.8s to go
on time step 17552 (time=175.52)

FloatProgress(value=0.0, description='0% done ', max=2090.0)

Meep progress: 19.78/2090.0 = 0.9% done in 4.0s, 418.8s to go
on time step 1979 (time=19.79), 0.00202214 s/step
Meep progress: 39.53/2090.0 = 1.9% done in 8.0s, 415.1s to go
on time step 3954 (time=39.54), 0.00202605 s/step
Meep progress: 59.38/2090.0 = 2.8% done in 12.0s, 410.5s to go
on time step 5939 (time=59.39), 0.00201567 s/step
Meep progress: 79.18/2090.0 = 3.8% done in 16.0s, 406.5s to go
on time step 7920 (time=79.2), 0.00202013 s/step
Meep progress: 98.98/2090.0 = 4.7% done in 20.0s, 402.4s to go
on time step 9900 (time=99), 0.00202077 s/step
Meep progress: 118.74000000000001/2090.0 = 5.7% done in 24.0s, 398.6s to go
on time step 11877 (time=118.77), 0.00202426 s/step
Meep progress: 138.52/2090.0 = 6.6% done in 28.0s, 394.6s to go
on time step 13855 (time=138.55), 0.00202228 s/step
Meep progress: 158.37/2090.0 = 7.6% done in 32.0s, 390.4s to go
on time step 15840 (time=158.4), 0.00201569 s/step
Meep progress: 178.15/2090.0 = 8.5% done in 36.0s, 386.4s to go
on time step 17819

In [1]:
plt.figure(dpi=150)
freqs = np.linspace(0,30e15,1000)
plt.plot(freqs,Efavg*tsim*eps0*4*np.pi*3/2**21,'g''.')
#plt.xlim(4e15,20e15)
plt.yscale("log")
#plt.ylim(0.1e-28,4000e-28)
plt.grid()
#plt.savefig('10_7ps.png')
plt.show()

#np.savez("10_7ps.npz", Efavg_30000 = Efavg)

NameError: name 'plt' is not defined